**Unit 2 - Part 1a: LangChain Setup & Models**

In [ ]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 14.6 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key:")

Enter your Google API key:··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Model A: The "Accountant" (Precision)
llm_focused = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

#Modek B: The
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0)

In [ ]:
prompt = "Define the word 'Idea' in one sentence."

print("--- FOCUSED (Temp=0) ---")
print(f"Run 1: {llm_focused.invoke(prompt).content}")
print(f"Run 2: {llm_focused.invoke(prompt).content}")

--- FOCUSED (Temp=0) ---
Run 1: An idea is a thought, concept, or mental image formed in the mind.
Run 2: An idea is a thought, concept, or suggestion that is formed or exists in the mind.


In [ ]:
print("--- CREATIVE (Temp=1) ---")
print(f"Run 1: {llm_creative.invoke(prompt).content}")
print(f"Run 2: {llm_creative.invoke(prompt).content}")

--- CREATIVE (Temp=1) ---
Run 1: An idea is a thought, suggestion, or concept that arises in the mind, often as a potential solution, plan, or understanding.
Run 2: An idea is a thought, concept, or plan that exists in the mind.


**Unit 2 - Part 1b: Prompts & Parsers**

In [ ]:
#Setup from Part 1a (Hidden for brevity)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key:")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
#System message = controls tone + behavior
from langchain_core.messages import SystemMessage, HumanMessage

#Scenario: Make the AI rude.

messages = [
    SystemMessage(content="You are a friendly geography tutor who explains answers clearly and simply."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print(response.content)

That's a great question!

The capital of France is **Paris**.

It's a beautiful city, famous for landmarks like the Eiffel Tower, the Louvre Museum, and Notre Dame Cathedral. Have you ever heard of it before?


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate {input_language} to {output_language}."),
    ("human", "{text}")
])

# We can check what inputs it expects
print(f"Required variables: {template.input_variables}")

Required variables: ['input_language', 'output_language', 'text']


In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Raw Message
raw_msg = llm.invoke("Hi")
print(f"Raw Type: {type(raw_msg)}")

# Parsed String
clean_text = parser.invoke(raw_msg)
print(f"Parsed Type: {type(clean_text)}")
print(f"Content: {clean_text}")

Raw Type: <class 'langchain_core.messages.ai.AIMessage'>
Parsed Type: <class 'str'>
Content: Hello! How can I help you today?


In [ ]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter your Google API Key:")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
template = ChatPromptTemplate.from_template("Tell me a fun fact about {topic}")
parser = StrOutputParser()

**Unit 2 - Part 1c: LCEL (LangChain Expression Language)**

In [ ]:
prompt_value = template.invoke({"topic": "Ice cream"})

response_obj = llm.invoke(prompt_value)

final_text = parser.invoke(response_obj)

print(final_text)

Here's a fun fact about ice cream:

The **ice cream cone** was popularized, if not outright invented, by accident at the **1904 St. Louis World's Fair**!

An ice cream vendor ran out of dishes, and a Syrian waffle vendor next door, named Ernest Hamwi, rolled his crispy waffles into cones to help out. This ingenious solution became an instant hit and revolutionized how people enjoyed their ice cream!


In [ ]:
chain = template | llm | parser

print(chain.invoke({"topic": "Octopuses"}))

Here's a fun fact about octopuses:

Octopuses have **three hearts** and **blue blood!**

Two hearts pump blood through their gills, and a third, larger heart circulates blood to the rest of their body. Their blood is blue because it uses a copper-based protein called hemocyanin to transport oxygen, instead of the iron-based hemoglobin that makes our blood red.


**Assignment**

Create a chain that:

Takes a movie name.
Asks for its release year.
Calculates how many years ago that was (You can try just asking the LLM to do the math).
Try to do it in one line of LCEL.

In [ ]:
template = ChatPromptTemplate.from_template(
    "Tell me the release year of the movie '{movie}'. "
    "Assume the current year is {current_year}. "
    "Return EXACTLY in this format:\n"
    "Movie Name: {movie}\n"
    "{movie} was released in YEAR. It was released X years ago."
)


In [ ]:
chain = template | llm | parser

print(chain.invoke({
    "movie": "Inception",
    "current_year": 2026
}))


Movie Name: Inception
Inception was released in 2010. It was released 16 years ago.
